In [18]:
import pandas as pd
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter  # 专门提供经纬度的
from cartopy.util import add_cyclic_point
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib.patheffects as path_effects
import matplotlib.path as mpath
from cnmaps import get_adm_maps, draw_maps
from matplotlib import ticker
import cmaps
from matplotlib.ticker import MultipleLocator, FixedLocator
from eofs.standard import Eof
from scipy.ndimage import filters
from tqdm import tqdm
import geopandas as gpd
import salem
from toolbar.TN_WaveActivityFlux import TN_WAF_3D, TN_WAF
from toolbar.curved_quivers.modplot import Curlyquiver
import pprint


# 数据读取
year = [1961, 2022]
try:
    # 读取相关系数
    sat = xr.open_dataset(r'D:\PyFile\paper1\cache\reg_lbm\sat.nc')
    sst = xr.open_dataset(r"E:\data\NOAA\ERSSTv5\sst.mnmean.nc")
    wh = xr.open_dataset(r'D:\PyFile\paper1\cache\reg_lbm\wh.nc')
    gpcp = xr.open_dataset(r'D:\PyFile\paper1\cache\reg_lbm\gpcp.nc')
    olr = xr.open_dataset(r"D:\PyFile\paper1\cache\olr\olr_same.nc")
except:
    # 读取数据
    sat = xr.open_dataset(r"E:\data\ERA5\ERA5_singleLev\ERA5_sgLEv.nc")
    sst = xr.open_dataset(r"E:\data\NOAA\ERSSTv5\sst.mnmean.nc")
    wh = xr.open_dataset(r"E:\data\ERA5\ERA5_pressLev\era5_pressLev.nc")
    gpcp = xr.open_dataset(r"E:\data\NOAA\PREC\precip.mon.anom.nc")
    olr = xr.open_dataset(r"D:\PyFile\paper1\cache\olr\olr_same.nc")
    # 数据描述信息修改
    change = sat.sel(   date=slice(str(year[0] - 1) + '-01-01', str(year[1] + 1) + '-12-31'),
                        latitude=[90 - i * 0.5 for i in range(361)], longitude=[i * 0.5 for i in range(720)])['t2m']
    sat = xr.DataArray(change.data, coords=[('time', pd.to_datetime(change['date'], format="%Y%m%d")),
                                            ('lat', change['latitude'].data),
                                            ('lon', change['longitude'].data)]).to_dataset(name='t2m')
    sst = sst.sel(  time=slice(str(year[0] - 1) + '-01-01', str(year[1] + 1) + '-12-31'),
                    latitude=[90 - i * 2 for i in range(89)], longitude=[i * 2 for i in range(180)])['sst']
    change_u = wh.sel(   date=slice(str(year[0] - 1) + '-01-01', str(year[1] + 1) + '-12-31'),
                         pressure_level=[200, 300, 400, 500, 600, 700, 850],
                        latitude=[90 - i * 0.5 for i in range(361)], longitude=[i * 0.5 for i in range(720)])['u']
    change_v = wh.sel(   date=slice(str(year[0] - 1) + '-01-01', str(year[1] + 1) + '-12-31'),
                         pressure_level=[200, 300, 400, 500, 600, 700, 850],
                        latitude=[90 - i * 0.5 for i in range(361)], longitude=[i * 0.5 for i in range(720)])['v']
    change_z = wh.sel(   date=slice(str(year[0] - 1) + '-01-01', str(year[1] + 1) + '-12-31'),
                         pressure_level=[200, 300, 400, 500, 600, 700, 850],
                        latitude=[90 - i * 0.5 for i in range(361)], longitude=[i * 0.5 for i in range(720)])['z']
    wh = xr.Dataset({  'u': (['time', 'level', 'lat', 'lon'], change_u.data),
                                'v': (['time', 'level', 'lat', 'lon'], change_v.data),
                                'z': (['time', 'level', 'lat', 'lon'], change_z.data)},
                      coords={  'time': pd.to_datetime(change_u['date'], format="%Y%m%d"),
                                'level': change_u['pressure_level'].data,
                                'lat': change_u['latitude'].data,
                                'lon': change_u['longitude'].data})
    sat.to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\sat.nc')
    wh.to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\wh.nc')
    gpcp.to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\gpcp.nc')
# 数据切片
olr = olr['olr']
pre = gpcp['precip'].sel(time=slice(f'{year[0]}-01-01', f'{year[1]}-12-31'))
pre = pre.sel(time=pre.time.dt.month.isin([7, 8]))
sst = sst['sst'].sel(time=slice(f'{year[0]}-01-01', f'{year[1]}-12-31'))
sst = sst.sel(time=sst.time.dt.month.isin([7, 8]))
T2m = sat['t2m'].sel(time=slice(f'{year[0]}-01-01', f'{year[1]}-12-31'))
T2m_78 = T2m.sel(time=T2m.time.dt.month.isin([7, 8]))
# 经纬度
lon_pre = pre['lon']
lat_pre = pre['lat']
lon_sst = sst['lon']
lat_sst = sst['lat']
lon_t2m = T2m['lon']
lat_t2m = T2m['lat']
lon_olr = olr['lon']
lat_olr = olr['lat']

# 将七八月份数据进行每年平均
lon1, lon2, lat1, lat2 = 40, 82.5, 50, 72
sat_78 = T2m_78.groupby('time.year').mean('time')
sat_78 = sat_78.sel(lat=slice(lat2, lat1), lon=slice(lon1, lon2))
sat_78 = sat_78.mean(['lat', 'lon'])
sat_78 = np.array(sat_78)
sat_detrend = np.polyfit(np.arange(len(sat_78)), sat_78[:],1)
sat_detrend = np.polyval(sat_detrend, np.arange(len(sat_78)))
sat_78 = sat_78 - sat_detrend
pre_78 = pre.groupby('time.year').mean('time')
sst_78 = sst.groupby('time.year').mean('time')
t2m_78 = T2m.groupby('time.year').mean('time')
pre_78 = np.array(pre_78)
sst_78 = np.array(sst_78)
t2m_78 = np.array(t2m_78)
olr_78 = np.array(olr)

In [29]:
reg_sst = [[np.polyfit(sat_78[:], sst_78[:, ilat, ilon], 1)[0] if not np.isnan(sst_78[:, ilat, ilon]).any() else np.nan for ilon in range(len(lon_sst))] for ilat in tqdm(range(len(lat_sst)), desc='计算LBM 2mT sst', position=0, leave=True)]
xr.DataArray(reg_sst, coords=[lat_sst, lon_sst], dims=['lat', 'lon']).to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\reg_lbm_t2m_sst.nc')

计算LBM 2mT sst:   3%|▎         | 3/89 [00:42<20:26, 14.26s/it]


PermissionError: [Errno 13] Permission denied: 'D:\\PyFile\\paper1\\cache\\reg_lbm\\reg_lbm_t2m_sst.nc'

In [31]:
xr.open_dataset(r'D:\PyFile\paper1\cache\reg_lbm\reg_lbm_t2m_sst.nc')['__xarray_dataarray_variable__']

-0.4275289820089265

In [32]:
reg_t2m = [[np.polyfit(sat_78[:], t2m_78[:, ilat, ilon], 1)[0] for ilon in range(len(lon_t2m))] for ilat in tqdm(range(len(lat_t2m)), desc='计算LBM 2mT t2m', position=0, leave=True)]
xr.DataArray(reg_t2m, coords=[lat_t2m, lon_t2m], dims=['lat', 'lon']).to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\reg_lbm_t2m_t2m.nc')

计算LBM 2mT t2m: 100%|██████████| 361/361 [00:13<00:00, 27.40it/s]


In [33]:
reg_olr = [[np.polyfit(sat_78[:], olr[:, ilat, ilon], 1)[0] for ilon in range(len(lon_olr))] for ilat in tqdm(range(len(lat_olr)), desc='计算LBM 2mT olr', position=0, leave=True)]
xr.DataArray(reg_olr, coords=[lat_olr, lon_olr], dims=['lat', 'lon']).to_netcdf(r'D:\PyFile\paper1\cache\reg_lbm\reg_lbm_t2m_olr.nc')

计算LBM 2mT olr: 100%|██████████| 361/361 [01:11<00:00,  5.06it/s]
